# IMPORT PYTHON MODULES

In [1]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

# IMPORT AND VALIDATE DATA 

In [2]:
warnings.filterwarnings("ignore")

#import data
data = r'conversion_paths.csv'
cp = pd.read_csv(data)

cp.head()

,user_pseudo_id,medium_path,conversion_flag,first_touchpoint,conversion_timestamp
0,flN8vT0gmTR1v8Ixommd1+u8s4ZvO8No7p7wl0X1v6Y+4z...,NaN,0,2022-12-28 00:00:01.914383 UTC,NaN
1,rgJvzjwn131LAGM5p0MZLCniNp8xX9VnGk861Dgu1wXNAA...,NaN,0,2022-12-28 00:00:04.792183 UTC,NaN
2,+JIUpNSpBdH11qifFKE/DwVZ5jozE6X2jJULmP3SZ0APUl...,NaN,0,2022-12-28 00:00:33.063 UTC,NaN
3,xO4dP3GBXDpOxHNwFIDCBnMKI1cBhsdZH+Eru7eyEoc0o8...,NaN,1,2022-12-28 00:00:52.439504 UTC,2023-01-23 14:16:55.060907 UTC
4,nP78ikNKi2iu1IJUO+LWIPcoByh3ZWFvlIzIl174i1jCuK...,paidsocial,0,2022-12-28 00:00:52.722434 UTC,NaN


In [3]:
cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173982 entries, 0 to 173981
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_pseudo_id        173982 non-null  object
 1   medium_path           71009 non-null   object
 2   conversion_flag       173982 non-null  int64 
 3   first_touchpoint      173982 non-null  object
 4   conversion_timestamp  5475 non-null    object
dtypes: int64(1), object(4)
memory usage: 6.6+ MB


# CLEANING DATA

In [4]:
# Remove users without medium path info
data_filtered = cp[cp['medium_path'].notnull()]
data_filtered['first_touchpoint'] = pd.to_datetime(data_filtered['first_touchpoint'])
data_filtered['conversion_timestamp'] = pd.to_datetime(data_filtered['conversion_timestamp'])

In [5]:
data_filtered.head()

,user_pseudo_id,medium_path,conversion_flag,first_touchpoint,conversion_timestamp
4,nP78ikNKi2iu1IJUO+LWIPcoByh3ZWFvlIzIl174i1jCuK...,paidsocial,0,2022-12-28 00:00:52.722434+00:00,NaT
5,Kkx/yH2k8X5rWdx+wG7DPuxpEyCsvSNwDqr4BzWGpxQw38...,cpc,0,2022-12-28 00:01:01.875587+00:00,NaT
6,brAeKj/gaKj8RkzQBIJfe/5XDKShTLwOcTh6yXQink4Kuo...,organic,0,2022-12-28 00:01:15.005975+00:00,NaT
9,4i0jYrhlk2CNR94hRIat8WPFLkU8dRdcPgKA1oDlwsWbUe...,cpc,0,2022-12-28 00:02:08.431349+00:00,NaT
10,JDClr9MBvuQkNi/PeCvCFScJ0NL/3qIlq0ZWVYzoGSbsNC...,paidsocial,0,2022-12-28 00:02:24.791342+00:00,NaT


# PREPROCESS DATA

In [6]:
converted_users = data_filtered[data_filtered['conversion_flag'] == 1]

expanded_paths_conv_user = converted_users['medium_path'].str.split(',', expand=True)

not_converted_users = data_filtered[data_filtered['conversion_flag'] == 0]

expanded_paths_non_conv_user = not_converted_users['medium_path'].str.split(',', expand=True)



In [16]:
# Define the maximum number of steps
# Why 13? 99% of users have max 13 steps, see details 'data_analysis'
MAX_STEPS = 13


# Function to split and expand medium paths
def expand_medium_path(path):
    if pd.notnull(path):
        steps = path.split(',')
        return pd.Series(steps)
    else:
        return pd.Series([None])

# Apply the function to the medium_path column
expanded_paths = data_filtered['medium_path'].apply(expand_medium_path)

expanded_paths_first_5 = expanded_paths.iloc[:, :MAX_STEPS]

final_df = pd.concat([data_filtered, expanded_paths_first_5], axis=1)


In [18]:
# Create a list of column names
column_names = ['user_pseudo_id', 'medium_path', 'conversion_flag', 'first_touchpoint', 'conversion_timestamp']
for i in range(1, MAX_STEPS + 1):
    column_names.append(f'step{i}')

# Assign the column names to the DataFrame
final_df.columns = column_names

In [14]:
final_df.head(15)

,user_pseudo_id,medium_path,conversion_flag,first_touchpoint,conversion_timestamp,step1,step2,step3,step4,step5,step6,step7,step8,step9,step10,step11,step12,step13
4,nP78ikNKi2iu1IJUO+LWIPcoByh3ZWFvlIzIl174i1jCuK...,paidsocial,0,2022-12-28 00:00:52.722434+00:00,NaT,paidsocial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Kkx/yH2k8X5rWdx+wG7DPuxpEyCsvSNwDqr4BzWGpxQw38...,cpc,0,2022-12-28 00:01:01.875587+00:00,NaT,cpc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,brAeKj/gaKj8RkzQBIJfe/5XDKShTLwOcTh6yXQink4Kuo...,organic,0,2022-12-28 00:01:15.005975+00:00,NaT,organic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4i0jYrhlk2CNR94hRIat8WPFLkU8dRdcPgKA1oDlwsWbUe...,cpc,0,2022-12-28 00:02:08.431349+00:00,NaT,cpc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,JDClr9MBvuQkNi/PeCvCFScJ0NL/3qIlq0ZWVYzoGSbsNC...,paidsocial,0,2022-12-28 00:02:24.791342+00:00,NaT,paidsocial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,+kb+nwGF9iTcHXXgvHszlWeQLO1dxXlncw4YLR3JEBVJyj...,paidsocial,0,2022-12-28 00:03:45.109434+00:00,NaT,paidsocial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,jNPY7CFHQQ357ZqBTiMlYNfX2FFoGs9AIBTEG4SKP54Dlg...,gotoweb,0,2022-12-28 00:04:45.740631+00:00,NaT,gotoweb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,RB8Yadge42beQcpxhZQlgHTPCnFDydSgrZapYMdWtNkHLg...,paidsocial,0,2022-12-28 00:05:38.993307+00:00,NaT,paidsocial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,xSj9j9Y/Van7PMtBpdN0zTCx0/gqzV5hFA5wy9kswCB1CZ...,"cpc,email,cpc",0,2022-12-28 00:06:03.526154+00:00,NaT,cpc,email,cpc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Rk/Lfpr8aLlShdqmGheoaqU7aydKIskNFf91ovfFMniU5t...,"organic,organic,organic,organic,organic,organi...",0,2022-12-28 00:06:29.739224+00:00,NaT,organic,organic,organic,organic,organic,organic,organic,organic,organic,organic,organic,organic,organic


# STORE FINAL DATASET

In [19]:
final_df.to_csv('final_dataset.csv')